In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import pandas as pd

Using TensorFlow backend.


In [2]:
t=time.time()
train_data = np.array(pd.read_csv("train.csv",header=None)).astype("float32")
train_labels = np.array(pd.read_csv("train_label.csv",header=None)).astype("int")
#eval_data = np.array(list(np.loadtxt("eval.csv", delimiter=","))).astype("float32")
#eval_labels = np.array(list(np.loadtxt("eval_label.csv", delimiter=","))).astype("int")
print(time.time()-t)

263.6854102611542


In [3]:
testset1 = np.array(pd.read_csv("Fine1.csv",header=None)).astype("float32")
testset2 = np.array(pd.read_csv("Fine2.csv",header=None)).astype("float32")
print(np.shape(testset2))

(123220, 1728)


In [5]:
testset1=np.array(testset1).astype("float32")

In [4]:
x_train = np.reshape(train_data,(-1, 24,24,3))
#x_val = eval_data.reshape(-1, 24,24,3)
y_train = np.reshape(train_labels,(-1,1))
#y_val = eval_labels.reshape(-1,1)

y_train = to_categorical(y_train)
#y_val = to_categorical(y_val)

#print(np.shape(x_train),np.shape(x_val),np.shape(y_train),np.shape(y_val))
print(np.shape(x_train),np.shape(y_train))
print(y_train[0])

IndexError: index -110 is out of bounds for axis 1 with size 107

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (6, 6), activation='relu',
                 input_shape = (24, 24, 3)))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
#model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
#model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [ ]:
hist = model.fit(x=x_train, y=y_train, batch_size=1024,
                           epochs=10, #Increase this when not on Kaggle kernel
                           verbose=2,  #1 for ETA, 0 for silent
                           callbacks=[annealer])

In [ ]:
#final_loss, final_acc = model.evaluate(x_val, y_val, verbose=0)
#print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
plt.plot(hist.history['loss'], color='b')
#plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
#plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
'''y_hat = model.predict(x_val)
print(np.shape(y_hat))
y_pred = np.argmax(y_hat, axis=1)
y_true = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_true, y_pred)
print(cm)'''

In [ ]:
x_test = np.reshape(testset1,(-1, 24,24,3))
y_hat1 = model.predict(x_test)
np.savetxt('spsk.csv',y_hat1,delimiter=',')
print(np.shape(x_test))
x_test = np.reshape(testset2,(-1, 24,24,3))
y_hat2 = model.predict(x_test)
np.savetxt('skfm.csv',y_hat2,delimiter=',')
print(np.shape(x_test))


In [ ]:
#y_hat1=np.loadtxt('spsk.csv',delimiter=',')
y_hat2=np.loadtxt('skfm.csv',delimiter=',')
y_hat2=y_hat2[:82820,:]

p=np.array(100*y_hat1,dtype=int)
print(np.shape(p))
t=np.zeros((51,3))
x = np.arange(1.2,1.71,0.01)
for i in range(51):
    for j in range(3):
        t[i,j]=np.mean(p[i*2020:(i+1)*2020,j])

plt.figure()
plt.plot(x,t[:,0],color='r')
plt.plot(x,t[:,1],color='g')
plt.plot(x,t[:,2],color='b')
plt.legend(['Spiral', 'Skyrmion', 'Ferromagnet'])
plt.title('SpSk')
plt.xlabel('B')
plt.ylabel('Phase Prediction %')

plt.show()

p=np.array(100*y_hat2,dtype=int)
print(np.shape(p))
t=np.zeros((41,3))
x = np.arange(2.8,3.2,0.01)
for i in range(41):
    for j in range(3):
        t[i,j]=np.mean(p[i*2020:(i+1)*2020,j])

plt.plot(x,t[:,0],color='r')
plt.plot(x,t[:,1],color='g')
plt.plot(x,t[:,2],color='b')
plt.legend(['Spiral', 'Skyrmion', 'Ferromagnet'])
plt.title('SkFm')
plt.xlabel('B')
plt.ylabel('Phase Prediction %')

plt.show()